In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim

from clients_attackers import *
from server import Server
from tasks import mnist

In [2]:
# FL training configurations
AR='fedavg' ## aggregation rule
loader_type='dirichlet'
loader_path='./data'
num_clients=10
test_batch_size=64
lr=0.01
momentum=0.5
epochs=30
inner_epochs=1
device='cuda'


### Specifying the data and model for the Federated Learning task

In [3]:
trainData = mnist.train_dataloader(num_clients, loader_type=loader_type, path=loader_path,
                                    store=False)
testData = mnist.test_dataloader(test_batch_size)
Net = mnist.Net
criterion = F.cross_entropy

Initialize a data loader


### Configuring the server

In [4]:
model0 = Net()
server = Server(model0, testData, criterion, device)
server.set_AR(AR)

### Configuring the clients and attach them to the server


In [5]:
list_of_backdoor_attacker=[1,3,5]
 # i is the client ID
for i in range(num_clients):
        model = Net()
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

        if i in list_of_backdoor_attacker:           
            client_i = Attacker_Backdoor(i, model, trainData[i], optimizer, criterion, device, inner_epochs)
        else:
            client_i = Client(i, model, trainData[i], optimizer, criterion, device, inner_epochs)
        server.attach(client_i)

### Sanity check of the pipeline

In [6]:
loss, accuracy = server.test()
aloss, asr     = server.test_backdoor()
steps=0
print(f"[{steps}]\t  Loss:\t{ loss:.4f}\t\tAccuracy:\t{accuracy:.2f}") 
print(f"[{steps}]\tA.Loss:\t{aloss:.4f}\t\t     ASR:\t{asr:.2f}\n\n") 

[Server] Start testing

Test set: Average loss: 2.3059, Accuracy: 560/10000 (6%)

[Server] Start testing backdoor

Test set (Backdoored): Average loss: 2.2624, Success rate: 79/10000 (1%)

[0]	  Loss:	2.3059		Accuracy:	5.60
[0]	A.Loss:	2.2624		     ASR:	0.79




### Run the training for the Federated learning task

In [7]:
for j in range(epochs):
    steps = j + 1
    server.distribute()
    #         group=Random().sample(range(5),1) # random subset sampling was not used in our paper
    group = range(num_clients) # we assume all clients are involved in synchronization
    server.train(group)

    loss, accuracy = server.test()
    aloss, asr     = server.test_backdoor()

    print(f"[{steps}]\t  Loss:\t{ loss:.4f}\t\tAccuracy:\t{accuracy:.2f}") 
    print(f"[{steps}]\tA.Loss:\t{aloss:.4f}\t\t     ASR:\t{asr:.2f}\n\n") 

[Server] Start testing

Test set: Average loss: 2.2952, Accuracy: 1032/10000 (10%)

[Server] Start testing backdoor

Test set (Backdoored): Average loss: 2.0398, Success rate: 10000/10000 (100%)

[1]	  Loss:	2.2952		Accuracy:	10.32
[1]	A.Loss:	2.0398		     ASR:	100.00


[Server] Start testing

Test set: Average loss: 2.2397, Accuracy: 1032/10000 (10%)

[Server] Start testing backdoor

Test set (Backdoored): Average loss: 1.5253, Success rate: 10000/10000 (100%)

[2]	  Loss:	2.2397		Accuracy:	10.32
[2]	A.Loss:	1.5253		     ASR:	100.00


[Server] Start testing

Test set: Average loss: 1.7479, Accuracy: 3029/10000 (30%)

[Server] Start testing backdoor

Test set (Backdoored): Average loss: 1.4667, Success rate: 7233/10000 (72%)

[3]	  Loss:	1.7479		Accuracy:	30.29
[3]	A.Loss:	1.4667		     ASR:	72.33


[Server] Start testing

Test set: Average loss: 0.9148, Accuracy: 7455/10000 (75%)

[Server] Start testing backdoor

Test set (Backdoored): Average loss: 2.2911, Success rate: 2214/10000 (22